In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
device = torch.device("cuda")
print(device)

cuda


In [3]:
block_size = 64
batch_size = 128
max_iters = 3000
learning_rate = 3e-3
eval_iters = 500
n_embd = 384
n_head = 4
n_layer = 4
dropout=0.2

In [6]:
chars = ""
with open('/content/wizard_of_oz.txt','r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))

#chars = sorted(set(text))
print(chars)
vocab_size=len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
80


In [7]:
#Tokenziers
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s:[string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text),dtype=torch.long)

In [8]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    #print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device),y.to(device)
    return x,y

In [10]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits,loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [11]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd,head_size,bias=False)
        self.query = nn.Linear(n_embd,head_size,bias=False)
        self.value = nn.Linear(n_embd,head_size,bias=False)
        self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self,x):

        #input of size (batch,time-step, channels)
        #output of size (batch,time-steo,head_size)
        B,T,C = x.shape

        k = self.key(x)    #(B,T,hs)
        q = self.query(x)  #(B,T,hs)

        #compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B,T,hs) @ (B,hs,T) --> (B,T,T)
        wei = wei.masked_fill(self.tril[:T,:T]==0,float('-inf')) #(B,T,T)
        wei = F.softmax(wei,dim=-1) # (B,T,T)
        wei = self.dropout(wei)

        #perform the weighted aggregation of the values
        v = self.value(x)
        out = wei @ v # (B,T,T) @ (B,T,hs) --> (B,T,hs)

        return out


In [12]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel"""

    def __init__(self, num_heads,head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads,n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self,x):
        out = torch.cat([h(x) for h in self.heads],dim=-1) # (B,T,F) -> (B,T,[h1,h1,h1,h1,h2,h2,h2,h2,h3,h3,h3,h3])
        out = self.dropout(self.proj(out))
        return out

In [13]:
class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity"""

    def __init__(self,n_embd):
        super().__init__()
        self.net = nn.Sequential(
                        nn.Linear(n_embd, 4 * n_embd),
                        nn.ReLU(),
                        nn.Linear(4 * n_embd, n_embd),
                        nn.Dropout(dropout)
                    )
    def forward(self,x):
        return self.net(x)

In [14]:
class Block(nn.Module):
    """Transformer block: communication followed by computation"""

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head,head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)


    def forward(self,x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x+y)

        return x

In [15]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
        self.position_embedding_table = nn.Embedding(block_size,n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd,n_head=n_head) for _ in range(n_layer)])

        self.ln_f = nn.LayerNorm(n_embd) #Final layer norm
        self.lm_head = nn.Linear(n_embd,vocab_size)

        self.apply(self._init_weights)


    def _init_weights(self,module):
        if isinstance(module,nn.Linear):
            torch.nn.init.normal_(module.weight,mean=0.0,std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight,mean=0.0,std=0.02)

    def forward(self,index,targets=None):

        B,T = index.shape
        #logits = self.token_embedding_table(index)
        #B, T, C = logits.shape
        #idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T,device=device))#(T,C)
        x = tok_emb + pos_emb #(B,T,C)
        x = self.blocks(x) #(B,T,C)
        x = self.ln_f(x) #(B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)


        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)

        return logits,loss

    def generate(self,index,max_new_tokens):
        #index is (B,T) array of indices in the current context

        for _ in range(max_new_tokens):
            #crop idx to the last block_size tokens
            index_cond = index[:,-block_size:]
            #get the predictions
            logits, loss = self.forward(index_cond)
            #focus only on the last time step
            logits = logits[:,-1,:] #becomes (B,C)
            #apply softmax to get probabilities
            probs = F.softmax(logits,dim=-1) # (B,C)
            #sample from the distribution
            index_next = torch.multinomial(probs,num_samples=1) #(B,1)
            #append sampled index to the running sequence
            index = torch.cat((index,index_next),dim=1) #(B,T+1)

        return index



In [16]:
model = GPTLanguageModel(vocab_size)
m = model.to(device)

In [17]:
#create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate)

for iter in range(max_iters):

    if iter%eval_iters==0:
            losses = estimate_loss()
            print(f"step:{iter}, train loss: {losses['train']:.4f}, val loss:{losses['val']:.4f}")

    #sample a batch of data
    xb,yb = get_batch("train")

    #evaluate the loss
    logits,loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step:0, train loss: 4.4783, val loss:4.4784
step:500, train loss: 3.1452, val loss:3.1598
step:1000, train loss: 3.1408, val loss:3.1540
step:1500, train loss: 3.1416, val loss:3.1520
step:2000, train loss: 3.1389, val loss:3.1515
step:2500, train loss: 3.1417, val loss:3.1531
3.129955530166626


In [20]:
context = torch.zeros((1,1,), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)


l.e
a ""esrrrt ld eoh rcoi weudsmooote5eco
o
c,wr a motWi t Lh,siasra Avti fabioirv .bitIaa daisncatDt h Bndhfnslt qoa  r 
r
qe tdem"hdmwl,fneubr o   r h sd hrhu deo',niks.d fe h n ,osrfne tcdan natcsssetp in la   hmuhnhnoi.nc ze.c eHcc
i a"ndod'b  u1ysrsrydne Tatruaro  nnduae  'btausspntpsWo,tiapo;T. ndedteb a
etc m anshdap,mowi  gf ssyfl an rut iia.g
lobtns  damslPere din .  tcl tav oFbgcwtatsea  yhtd eegeere  eab orh . sTu;bylesUorlmo ihrlnneaet niluhaursherg etch,niltsu  fst d
nne bngnngOemo
